# Эффект Джоуля-Томсона

In [47]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

matplotlib.use('pgf')
matplotlib.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

def read_files(files):
    data = []
    for f in files:
        data += [pd.read_csv(f, sep=',', skipinitialspace=True)]
    return data

T = [17, 30, 40, 50]
colors = ['red', 'green', 'blue', 'magenta']
[data] = read_files(['data/T.csv'])

def get_linear(x, y, xerr, yerr):
    a, b = np.polyfit(x, y, deg=1)
    aerr = np.sqrt(1 / (x.__len__() - 1) * (sum([(it - b)**2 for it in y]) 
                                                    / sum([it**2 for it in x]) - a**2))
    syserr = np.sqrt((np.average(xerr) / (max(x) - min(x)))**2
                         + (np.average(yerr) / (max(y) - min(y)))**2)
    a_syserr = a * syserr
    berr = aerr * np.sqrt(np.average([it**2 for it in x]))
    b_syserr = b * syserr
    return ((a, b), (np.sqrt(aerr**2 + a_syserr**2), np.sqrt(berr**2 + b_syserr**2)))

plt.figure(figsize=(7, 4))

mu = []
muerr = []
P_ERR = 0.1
# Аппроксимация
for i in range(1, 5):
    y = data[f'T{i}']
    yerr = data[f'dT{i}']
    z = np.linspace(min(data['P']), max(data['P']), 1000)
    ((a, b), (aerr, berr)) = get_linear(data['P'], y, [P_ERR], yerr)
    plt.plot(z, a * z + b, 
            label=f'$T = {T[i - 1]}$ \\textdegree C',
            color=colors[i - 1])
    mu.append(a)
    muerr.append(aerr)
    plt.errorbar(data['P'], data[f'T{i}'], xerr=P_ERR, yerr=yerr, 
                 fmt=f'{colors[i - 1][0]}.')

# Оформление
plt.xlabel('$\\Delta P$, A')
plt.ylabel('$\\Delta T$, \\textdegree C')
plt.grid(linestyle='--')
plt.legend()
plt.savefig('data/T.pgf')

inverse_T = [10**3 / (t + 273) for t in T]
mu_T = pd.DataFrame()
mu_T['$T$, K'] = T
mu_T['$1/T$, $10^3$ K$^{-1}$'] = inverse_T
mu_T['$\mu$'] = mu
mu_T['$\Delta\mu$'] = muerr
mu_T.to_csv('data/mu.csv')

R = 8.31
C_P = 7 / 2 * R

plt.figure(figsize=(7, 4))
((alpha, beta), (alpha_err, beta_err)) = get_linear(inverse_T, mu, [0], muerr)
z = np.linspace(min(inverse_T), max(inverse_T), 1000)
plt.plot(z, alpha * z + beta, label='', color='red')
coeffs = pd.DataFrame()
coeffs['a'] = [alpha * C_P * R / 2 / 100, alpha_err * C_P * R / 200, alpha_err / alpha * 100]
coeffs['b'] = [-beta * C_P / 10, beta_err * C_P / 10, -beta_err / beta * 100]
coeffs.to_csv('data/coeffs.csv')

plt.errorbar(inverse_T, mu, yerr=muerr, fmt='r.')
plt.xlabel('$T^{-1}$, $10^{-3}$ K$^{-1}$')
plt.ylabel('$\\mu$, $10^{-5}$')
plt.grid()
plt.savefig('data/mu.pgf')
